In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Libraries

In [ ]:
import pandas as pd
import numpy as np
import random
import time
import datetime

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS

import re
import json
from tqdm.autonotebook import tqdm
import string
import collections
from textblob import TextBlob

import spacy

import nltk
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn import preprocessing
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

from keras.preprocessing import sequence, text
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import utils
from keras.models import *
from keras.layers import *
from keras.callbacks import *
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers import Dense, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils.vis_utils import plot_model

import warnings
warnings.filterwarnings('ignore')

In [ ]:
#define stopwords
from nltk.corpus import stopwords

stopwords_list = stopwords.words('english') + list(string.punctuation)
stopwords_list += ["''", '""', '...', '``']

In [ ]:
def clean_text(txt):
     return re.sub('[^A-Za-z0-9.]+', ' ', str(txt).lower())

In [ ]:
#https://towardsdatascience.com/text-analysis-feature-engineering-with-nlp-502d6ea9225d

def text_cleaning(text, flg_stemm = False, flg_lemm = True, lst_stopwords = None):
    '''
    Converts all text to lower case, tokenize, remove multiple spaces, stopwords, stemming, lemmatize, 
    then convert all back to string
    
    text: string - name of column containing text
    lst_stopwords: list - list of stopwords to remove
    flg_stemm: bool - whether stemming is to be applied
    flg_lemm: bool - whether lemmitisation is to be applied
    '''
    
    #clean (convert to lowercase and remove punctuations and characters and then strip)
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
            
    #tokenize (convert from string to list)
    lst_text = text.split()
    
    #remove Stopwords
    if lst_stopwords is not None:
        lst_text = [word for word in lst_text if word not in 
                    stopwords_list]
                
    #stemming (remove -ing, -ly, ...)
    if flg_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        lst_text = [ps.stem(word) for word in lst_text]
                
    #lemmatisation (convert the word into root word)
    if flg_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        lst_text = [lem.lemmatize(word) for word in lst_text]
            
    #back to string from list
    text = " ".join(lst_text)
    return text

In [ ]:
MAX_LENGTH = 64
OVERLAP = 20
    
def shorten_sentences(sentences):
    """
    Sentences that have more than MAX_LENGTH words will be split
    into multiple sentences with overlappings.
    """
    short_sentences = []
    for sentence in sentences:
        words = sentence.split()
        if len(words) > MAX_LENGTH:
            for p in range(0, len(words), MAX_LENGTH - OVERLAP):
                short_sentences.append(' '.join(words[p:p+MAX_LENGTH]))
        else:
            short_sentences.append(sentence)
    return short_sentences

# OBTAIN

In [ ]:
#define paths
os.listdir('/kaggle/input/coleridgeinitiative-show-us-the-data/')
train_path = '../input/coleridgeinitiative-show-us-the-data/train'
test_path = '../input/coleridgeinitiative-show-us-the-data/test'

In [ ]:
#create a function to get the text from the JSON file and append it to the new column in table
def read_json_pub(filename, train_path = train_path, output = 'text'):
    json_path = os.path.join(train_path, (filename + '.json'))
    headings = []
    contents = []
    combined = []
    with open(json_path, 'r') as f:
        json_decode = json.load(f)
        for data in json_decode:
            headings.append(data.get('section_title'))
            contents.append(data.get('text'))
            combined.append(data.get('section_title'))
            combined.append(data.get('text'))
    
    all_headings = ' '.join(headings)
    all_contents = ' '.join(contents)
    all_data = '. '.join(combined)
    
    if output == 'text':
        return all_contents
    elif output == 'head':
        return all_headings
    else:
        return all_data

In [ ]:
#read 
train = pd.read_csv('../input/coleridgeinitiative-show-us-the-data/train.csv')

#review
train.head()

# Create Sentences & Labels

In [ ]:
#https://www.kaggle.com/jagdmir/spacy-ner-model

import nltk

DATA = []
label_count = 0

for idx,row in tqdm(train.iterrows()):
    pub = "../input/coleridgeinitiative-show-us-the-data/train/" + row.Id + ".json"            
    f = open(pub)  
    data = json.load(f)      
    
    balanced = False

    sentences = [clean_text(sentence) for sentence in nltk.sent_tokenize(str(data))]
    sentences = shorten_sentences(sentences) # make sentences short
    sentences = [sentence for sentence in sentences if len(sentence) > 10] # only accept sentences with length > 10 chars
    
    for sentence in sentences:          
      
        a = re.search(row.cleaned_label.lower(), sentence)
        b = re.search(row.dataset_label.lower(), sentence)
        c = re.search(row.dataset_title.lower(), sentence)
        cleaned_label = row.cleaned_label.lower()
        dataset_label = row.dataset_label.lower()
        dataset_title = row.dataset_title.lower()
        
        if  a != None:
            DATA.append((sentence, cleaned_label))
            label_count = label_count + 1
            balanced = True
        elif b != None:
            DATA.append((sentence, dataset_label))
            label_count = label_count + 1
            balanced = True
        elif c != None:
            DATA.append((sentence, dataset_title))
            label_count = label_count + 1
            balanced = True  
            
        else:
            pass
                
print("Text with dataset:", label_count)

In [ ]:
#get dataframe
train_df = pd.DataFrame(DATA)
train_df = train_df.rename({0: 'Sentence', 1: 'Label'}, axis = 1)

#review
train_df.tail(10)

In [ ]:
print('Sentence:', train_df['Sentence'][555])
print('\n')
print('Label:', train_df['Label'][555])

In [ ]:
print('Sentence:', train_df['Sentence'][50000])
print('\n')
print('Label:', train_df['Label'][50000])

# PREPROCESSING

### Train-Test-Split

In [ ]:
X = train_df['Sentence'].to_numpy()
y = train_df['Label'].to_numpy()

#split traing data into training a validation sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 123)

In [ ]:
#check shape
print('Train sentences:', X_train.shape, '\n', 
      'Test sentences:', X_test.shape, '\n', 
      'Train labels:', y_train.shape, '\n', 
      'Test labels:', y_test.shape)

### Tokenize

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

#limit on the number of features. We use the top 20K features
top_k = 20000

#limit on the length of text sequences. Sequences longer than this will be truncated
max_sequence_length = 500

#get max sequence length
max_length = len(max(X_train, key = len))
if max_length > max_sequence_length:
    max_length = max_sequence_length
    
max_vocab_length = 20000 # max number of words to have in our vocabulary

#method to count the unique words in vocabulary and assign each of those words to indices
tokenizer = Tokenizer(num_words = top_k)

#create vocabulary with training texts
tokenizer.fit_on_texts(list(X_train))

#convert text into integer sequences
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)
         
#fix sequence length to max value. 
#sequences shorter than the length are padded in the beginning and sequences longer are truncated at the beginning
#this turns our lists of integers into a 2D integer tensor of shape (samples, maxlen)
X_train_pad  = pad_sequences(X_train_seq, maxlen = max_length)
X_test_pad = pad_sequences(X_test_seq, maxlen = max_length)

In [ ]:
#number of unique words in the training data
size_of_vocabulary = len(tokenizer.word_index) + 1 #+1 for padding
print(size_of_vocabulary)

In [ ]:
word_index = tokenizer.word_index

### Encode Label

In [ ]:
from sklearn import preprocessing

#use the LabelEncoder to convert text labels to integers, 0, 1, 2, etc.
encoder = preprocessing.LabelEncoder()

#since we have two different data set (X_train and X_test), 
#we need to fit it on all of our data otherwise there might be some categories in the test set X_test that were not in the train set X_train 
#and we will get errors
encoder.fit(list(y_train) + list(y_test)) 
y_train = encoder.transform(y_train)
y_test = encoder.transform(y_test)

In [ ]:
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

In [ ]:
encoder.classes_

### Binarize Label

In [ ]:
num_classes = train_df['Label'].nunique() + 1
num_classes

In [ ]:
from keras import utils

#binarize the labels for the neural net
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test, num_classes)

In [ ]:
print('X_train shape:', X_train_pad.shape)
print('X_test shape:', X_test_pad.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

### Load Pretrained Word Vector

In [ ]:
#load the whole embedding into memory
embeddings_index = {}
f = open('../input/glove840b300dtxt/glove.840B.300d.txt')

for line in tqdm(f):
    values = line.split()
    word = values[0]
    try:
        coefs = np.asarray(values[1:], dtype = 'float32')
        embeddings_index[word] = coefs
    except ValueError: #catch the exception where there are strings in the GloVe text file, can be avoided if use glove.42B.300d.txt
        pass
    
f.close()

print('Found %s word vectors.' % len(embeddings_index))

In [ ]:
#create an embedding matrix for the words we have in the dataset
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in tqdm(word_index.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# MODELING

In [ ]:
#hyperparameters
filters = 4
kernel_size = 3
dropout_rate = 0.5
pool_size = 3

In [ ]:
#simple bidirectional LSTM with GloVe embeddings and Dense layers
sepcnn_model = Sequential()

#embedding layer
sepcnn_model.add(Embedding(size_of_vocabulary, 
                           300,
                           weights = [embedding_matrix], #load GloVe
                           input_length = X_train_pad.shape[0],
                           trainable = False)) #keep frozen

sepcnn_model.add(Dropout(rate = dropout_rate))
sepcnn_model.add(SeparableConv1D(filters = filters,
                                  kernel_size = kernel_size,
                                  activation = 'relu'))
sepcnn_model.add(SeparableConv1D(filters = filters,
                                  kernel_size = kernel_size,
                                  activation = 'relu'))
    
sepcnn_model.add(MaxPooling1D(pool_size = pool_size))
sepcnn_model.add(SeparableConv1D(filters = filters * 2,
                              kernel_size = kernel_size,
                              activation = 'relu'))

sepcnn_model.add(SeparableConv1D(filters = filters * 2,
                              kernel_size = kernel_size,
                              activation = 'relu'))
                 
sepcnn_model.add(GlobalAveragePooling1D())

sepcnn_model.add(Dropout(rate = dropout_rate))
                          
#output layer
sepcnn_model.add(Dense(num_classes, activation = 'softmax')) 

In [ ]:
#summary
sepcnn_model.summary()

In [ ]:
#plot
plot_model(sepcnn_model, to_file = 'sepcnn_model_plot.png', show_shapes = True, show_layer_names = True)

In [ ]:
#compile
sepcnn_model.compile(optimizer = keras.optimizers.Adam(0.0001), #low learning rate is good, but the model will take more iterations to converge
                    loss = 'categorical_crossentropy',
                    metrics = ['acc'])

In [ ]:
#from sklearn.utils import class_weight

#correct class imbalance
#class_weights = list(class_weight.compute_class_weight('balanced',
#                                                       np.unique(train_df['Label']),
#                                                       train_df['Label']))

#weights = {}
#for index, weight in enumerate(class_weights) : weights[index] = weight

In [ ]:
print('X_train shape:', X_train_pad.shape)
print('X_test shape:', X_test_pad.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

In [ ]:
#add callbacks
from keras.callbacks import EarlyStopping, ModelCheckpoint

#define the callbacks
early_stopping = [EarlyStopping(monitor = 'val_loss', patience = 5, verbose = 1),
                 ModelCheckpoint(filepath = 'sepcnn_model.h5', monitor = 'val_loss', save_best_only = True)]

In [ ]:
start = datetime.datetime.now()
batch_size = 256

#fit
sepcnn_history = sepcnn_model.fit(np.array(X_train_pad), np.array(y_train),
                                  #class_weight = weights,
                                  batch_size = batch_size,
                                  epochs = 20,
                                  validation_data = (np.array(X_test_pad), np.array(y_test)),
                                  steps_per_epoch = X_train_pad.shape[0] // 256,
                                  validation_steps = X_test_pad.shape[0] // 256,
                                  callbacks = early_stopping)

In [ ]:
end = datetime.datetime.now()
elapsed = end - start
print('Training took a total of {}'.format(elapsed))

In [ ]:
#save model
sepcnn_model.save('sepcnn_model.h5')

In [ ]:
fig , ax = plt.subplots(1,2)
fig.set_size_inches(20, 8)

sepcnn_train_acc = sepcnn_history.history['acc']
sepcnn_train_loss = sepcnn_history.history['loss']
sepcnn_val_acc = sepcnn_history.history['val_acc']
sepcnn_val_loss = sepcnn_history.history['val_loss']

epochs = range(1, len(sepcnn_train_acc) + 1)

ax[0].plot(epochs , sepcnn_train_acc , 'g-o' , label = 'Training Accuracy')
ax[0].plot(epochs , sepcnn_val_acc , 'y-o' , label = 'Validation Accuracy')
ax[0].set_title('SEPCNN Model Train & Validation Accuracy')
ax[0].legend(loc = 'lower right')
ax[0].set_xlabel('Epochs')
ax[0].set_ylabel('Accuracy')

ax[1].plot(epochs, sepcnn_train_loss , 'g-o' , label = 'Training Loss')
ax[1].plot(epochs, sepcnn_val_loss , 'y-o' , label = 'Validation Loss')
ax[1].set_title('SEPCNN Model Train & Validation Loss')
ax[1].legend()
ax[1].set_xlabel('Epochs')
ax[1].set_ylabel('Accuracy')

plt.show()

In [ ]:
#save
plt.savefig('sepcnn_acc_loss.png')

In [ ]:
print('Train loss & accuracy:', sepcnn_model.evaluate(X_train_pad, y_train))
print('\n')
print('Test loss & accuracy:', sepcnn_model.evaluate(X_test_pad, y_test))

In [ ]:
#make prediction
sepcnn_yhat_test = sepcnn_model.predict(X_test_pad)

#to evaluate accuracy we need a vector of labels
sepcnn_yhat_test = np.argmax(sepcnn_yhat_test, axis = 1)
sepcnn_y_test = np.argmax(y_test, axis = 1)

#get classification report
print('Model: SEPCNN', '\n', classification_report(sepcnn_y_test, sepcnn_yhat_test))

In [ ]:
preds = encoder.inverse_transform([np.argmax(sepcnn_yhat_test)]) 
preds

In [ ]:
#summary table
summary_table = pd.DataFrame({'Model': [],
                              'Accuracy': [],
                              'Precision': [], 'Recall': [], 'F1': []})

In [ ]:
#update summary table
summary_table.loc[0] = ['DL SEPCNN',
                        round(accuracy_score(sepcnn_y_test, sepcnn_yhat_test), 2),
                        round(precision_score(sepcnn_y_test, sepcnn_yhat_test, average = 'macro'), 2), 
                        round(recall_score(sepcnn_y_test, sepcnn_yhat_test, average = 'macro'), 2), 
                        round(f1_score(sepcnn_y_test, sepcnn_yhat_test, average = 'macro'), 2)]
summary_table.head()

In [ ]:
summary_table.to_csv('sepcnn_summary_table.csv')

# PREDICTION

In [ ]:
#get text
tqdm.pandas()
train['text'] = train['Id'].progress_apply(read_json_pub)

#clean text
train['text'] = train['text'].progress_apply(clean_text)

In [ ]:
from functools import partial

#read data
sample_submission = pd.read_csv('../input/coleridgeinitiative-show-us-the-data/sample_submission.csv')

#apply the function to submission data
tqdm.pandas()
sample_submission['text'] = sample_submission['Id'].progress_apply(partial(read_json_pub, train_path = test_path))

#review
sample_submission.head()

In [ ]:
temp_1 = [x.lower() for x in train['dataset_label'].unique()]
temp_2 = [x.lower() for x in train['dataset_title'].unique()]
temp_3 = [x.lower() for x in train['cleaned_label'].unique()]

existing_labels = set(temp_1 + temp_2 + temp_3)

In [ ]:
literal_matching = True
sepcnn_prediction = True

id_list = []
labels_list = []

for index, row in tqdm(sample_submission.iterrows()):

    sample_text = row['text']

    row_id = row['Id']
    
    #check if the sample text is equal to one of the train samples and if so, use those labels
    temp_df = train[train['text'] == clean_text(sample_text)]
    cleaned_labels = temp_df['cleaned_label'].to_list()
    
    #literal_matching for known label in sample text
    if literal_matching:
        for known_label in existing_labels:
            if known_label in sample_text.lower():    
                cleaned_labels.append(clean_text(known_label))
            
        print('cleaned label:', set(cleaned_labels))   
    
    #lstm_prediction 
    if sepcnn_prediction:
        
        #extract sentences
        sentences = [clean_text(sentence) for sentence in nltk.sent_tokenize(str(sample_text))]
        sentences = shorten_sentences(sentences) # make sentences short
        sentences = [sentence for sentence in sentences if len(sentence) > 10] # only accept sentences with length > 10 chars        
        
        tokenizer.fit_on_texts(list(sentence))
        sentence_seq = tokenizer.texts_to_sequences([sentence])
        sentence_pad  = pad_sequences(sentence_seq, maxlen = max_length)
            
        #predict
        sepcnn_labels = sepcnn_model.predict(sentence_pad)
    
        #get label
        sepcnn_labels = encoder.inverse_transform([np.argmax(sepcnn_labels)])
            
        print('sepcnn label:', set(sepcnn_labels))
        sepcnn_labels = set(sepcnn_labels)
        
    cleaned_labels += sepcnn_labels
    print('updated cleaned label:', set(cleaned_labels))
        
    cleaned_labels = set(cleaned_labels)
    cleaned_labels = [clean_text(x) for x in cleaned_labels]    
    labels_list.append('|'.join(cleaned_labels))
    print('label list:', labels_list)   
    id_list.append(row_id)
    print('\n')

In [ ]:
#get dataframe
sample_submission['PredictionString'] = labels_list
sample_submission.drop(columns = 'text', axis = 1, inplace = True)
sample_submission

In [ ]:
print(sample_submission['PredictionString'][0])
print('\n')
print(sample_submission['PredictionString'][1])
print('\n')
print(sample_submission['PredictionString'][2])
print('\n')
print(sample_submission['PredictionString'][3])

In [ ]:
#save
sample_submission.to_csv('submission.csv', index = False)

#check
submission = pd.read_csv('submission.csv')
submission